# Post-Processing (BATCH II)

#### Updated: March 20, 2023

#  

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
rootDir = 'L:/DATA/Alouette_I/BATCH_I_Run1/'
resultDir = rootDir + '05_result/'
logDir = rootDir + '06_log/'

#  

#### Generate list of unprocessed subdirectories, for further qualitative inspection:

In [3]:
df_process_log = pd.read_csv(logDir + 'process_log.csv')
df_p1 = df_process_log.sort_values('Process_timestamp', ascending=True)
df_p1 = df_p1.drop_duplicates(subset=['subdir_id'], keep='last') 
print(len(df_p1))
df_p1.head()

920


,Roll,Subdirectory,Images_processed,Images_dot,Images_num,Images_loss,Images_outlier,Process_time,Process_timestamp,User,subdir_id
792,R014207813,5031,416.0,0.0,117.0,247.0,52.0,286.101392,00:02.4,Rav Super3,R014207813_5031
573,R014207825,3760-38A,174.0,0.0,83.0,86.0,5.0,388.078494,00:05.8,Rav Super11,R014207825_3760-38A
723,R014207763,AUG-6-TO-AUG-13-1965,0.0,0.0,0.0,0.0,0.0,161.170390,00:06.2,Rav Super3,R014207763_AUG-6-TO-AUG-13-1965
20,R014207807,4953,0.0,0.0,0.0,0.0,0.0,186.827210,00:09.2,Rav Super11,R014207807_4953
654,R014207817,May-87,344.0,0.0,321.0,4.0,19.0,638.494545,00:10.3,Rav Super10,R014207817_4187-5


In [4]:
df1 = df_p1.loc[df_p1['Images_processed'] == 0]
print(len(df1))

529


In [5]:
df2 = df1[['Directory', 'Subdirectory', 'Images_processed']]
#df2 = df2.rename(columns={
#    'Roll': 'Directory'
#})
df2 = df2.sort_values(['Directory', 'Subdirectory']).reset_index(drop=True)
df2

,Directory,Subdirectory,Images_processed
0,R014207709,145,0.0
1,R014207709,146,0.0
2,R014207709,147,0.0
3,R014207709,148,0.0
4,R014207709,149,0.0
...,...,...,...
524,R014207825,3771-43A,0.0
525,R014207825,3779-43A-2,0.0
526,R014207825,3780-43A,0.0
527,R014207825,3783-69A-2,0.0


In [6]:
df2.to_csv(logDir + 'unprocessed_subdirs-BATCHII.csv', index=False)

In [7]:
df2.sample(10)

,Directory,Subdirectory,Images_processed
366,R014207810,5126,0.0
205,R014207766,OCT-22-TO-NOV-20-1964,0.0
419,R014207811,5091,0.0
132,R014207763,JULY-18-TO-AUG-2-1966,0.0
432,R014207811,5105,0.0
494,R014207817,May-79,0.0
373,R014207810,5136,0.0
482,R014207813,5046,0.0
288,R014207807,4929,0.0
405,R014207811,5076,0.0


#  

#### Identify outstanding subdirectories to process:

Check from process_log:

Look at duplicates:

In [ ]:
df_p1_ = df_process_log[df_process_log.duplicated(subset=['subdir_id'], keep=False)]
df_p1_ = df_p1_.sort_values(['subdir_id', 'Process_timestamp'])
print(len(df_p1_))
df_p1_.head(10)

Find any subdirectories that have not been processed:

In [ ]:
df_inventory = pd.read_csv(logDir + 'image_inventory.csv')
print(len(df_inventory))
df_inventory.head()

In [ ]:
subdir_ids_tot = df_inventory['subdir_id'].unique()
subdir_ids_proc = df_p1['subdir_id'].unique()
subdir_ids_rem = list(set(subdir_ids_tot) - set(subdir_ids_proc))
subdir_ids_rem

Check from resultDir:

In [ ]:
df_result = pd.DataFrame()
i = 0
for file in os.listdir(resultDir):
    if 'R' in file:
        directory = file
        for file2 in os.listdir(resultDir + directory + '/'):
            if 'result' in file2:
                fn_parts = file2.split('_')
                subdirectory = fn_parts[1].replace('.csv', '')
                try:
                    df_load = pd.read_csv(resultDir + directory + '/' + file2, sep=',')
                    n = len(df_load)
                except pd.errors.EmptyDataError:
                    n = 0
                row = pd.DataFrame({
                    'Directory': directory,
                    'Subdirectory': subdirectory,
                    'images_result': n,
                    'subdir_id': directory + '_' + subdirectory
                }, index=[i])
                df_result = pd.concat([df_result, row])
                i += 1
                if i % 100 == 0:
                    print(str(i) + ': Now inspecting...' + directory + '/' + subdirectory)

In [ ]:
print(len(df_result))
df_result.head()

In [ ]:
subdir_ids_tot = df_inventory['subdir_id'].unique()
subdir_ids_res = df_result['subdir_id'].unique()
subdir_ids_rem = list(set(subdir_ids_tot) - set(subdir_ids_res))
subdir_ids_rem

In [ ]:
df_p1_res = df_p1.merge(df_result, on=['Directory', 'Subdirectory', 'subdir_id'], how='left')
print(len(df_p1_res))
df_p1_res.head()

In [ ]:
df1 = df_p1_res.loc[df_p1_res['Images_processed'] == 0]
df2 = df_p1_res.loc[df_p1_res['images_result'] == 0]
df3 = df_p1_res.loc[(df_p1_res['Images_processed'] == 0) | (df_p1_res['images_result'] == 0)]
print(len(df1))
print(len(df2))
print(len(df3))
df3.head()

In [ ]:
#df3.to_csv(logDir + 'reprocess_list.csv', index=False)

Check mapped_coords:

In [ ]:
df_mc = pd.DataFrame()
i = 0
for file in os.listdir(resultDir + 'mapped_coords/'):
    if 'R' in file:
        directory = file
        for file2 in os.listdir(resultDir + 'mapped_coords/' + directory + '/'):
            subdirectory = file2
            n = 0
            for file3 in os.listdir(resultDir + 'mapped_coords/' + directory + '/' + subdirectory + '/'):
                if 'mapped_coords' in file3:
                    n += 1
            row = pd.DataFrame({
                'Directory': directory,
                'Subdirectory': subdirectory,
                'mapped_coords_result': n,
                'subdir_id': directory + '_' + subdirectory
            }, index=[i])
            df_mc = pd.concat([df_mc, row])
            i += 1
            if i % 100 == 0:
                    print(str(i) + ': Now inspecting...' + directory + '/' + subdirectory)

In [ ]:
print(len(df_mc))
df_mc.head()

In [ ]:
df_p1_mc = df_p1.merge(df_mc, on=['Directory', 'Subdirectory', 'subdir_id'], how='left')
print(len(df_p1_mc))
df_p1_mc.head()

In [ ]:
df1 = df_p1_mc.loc[df_p1_mc['Images_processed'] == 0]
df2 = df_p1_mc.loc[df_p1_mc['mapped_coords_result'] == 0]
df2a = df_p1_mc.loc[df_p1_mc['mapped_coords_result'].isnull()]
df3 = df_p1_mc.loc[(df_p1_mc['Images_processed'] == 0) | (df_p1_mc['mapped_coords_result'] == 0)]
print(len(df1))
print(len(df2))
print(len(df2a))
print(len(df3))
df3

In [ ]:
df4 = df_p1_mc.loc[(df_p1_mc['Images_processed'] != 0) & (df_p1_mc['mapped_coords_result'].isnull())]
print(len(df4))
df4

In [ ]:
#df4.to_csv(logDir + 'reprocess_list.csv', index=False)

#  

#### Concatenate results:

In [ ]:
df_result = pd.DataFrame()
i = 0
for file in os.listdir(resultDir):
    if 'R' in file:
        directory = file
        for file2 in os.listdir(resultDir + directory + '/'):
            if 'result' in file2:
                fn_parts = file2.split('_')
                subdirectory = fn_parts[1].replace('.csv', '')
                if i > 0:
                    if i % 100 == 0:
                        df_result = pd.read_csv(resultDir + 'result_total.csv')
                        print(len(df_result))
                try:
                    df_load = pd.read_csv(resultDir + directory + '/' + file2, sep=',')
                    n = len(df_load)
                except pd.errors.EmptyDataError:
                    n = 0
                df_result = pd.concat([df_result, df_load])
                i += 1
                if i % 100 == 0:
                    print('Now saving the ' + str(i) + 'th result...')
                    df_result.to_csv(resultDir + 'result_total.csv', index=False)

#  

#### Reduce columns:

In [ ]:
df_result = pd.read_csv(resultDir + 'result_total.csv')
print(len(df_result))
df_result.head()

In [ ]:
df_result = df_result[['Directory', 'Subdirectory', 'filename', 'processed_image_class', 'fmin', 'max_depth', 'year', 'day', 
                       'hour', 'minute', 'second', 'station_number', 'satellite_number', 'day_1', 'day_2', 'day_3', 
                       'hour_1', 'hour_2', 'minute_1', 'minute_2', 'second_1', 'second_2', 'station_number_1', 
                       'station_number_2', 'metadata_type', 'is_dot', 'func_name', 'limits']] 
df_result.head()

#  

#### Adjust year to baseline 1962:

In [ ]:
df_result['year'] = df_result['year'] + 1962

In [ ]:
df1 = df_result.copy(deep=True)
df1['count'] = 1
df1 = df1.groupby(['year']).sum()
df1 = df1.reset_index()
df1[['year', 'count']]

#  

#### Check day digits are < 10, reprocess day of the year:

In [ ]:
df_result[['day_1', 'day_2', 'day_3', 'day']].sample(10)

In [ ]:
sns.histplot(data=df_result, x='day_1')

In [ ]:
df_result['day_1'] = df_result['day_1'].apply(lambda x: np.nan if x>9 else x)
df_result['day_1'] = df_result['day_1'].astype('Int64')

In [ ]:
sns.histplot(data=df_result, x='day_1')

In [ ]:
sns.histplot(data=df_result, x='day_2')

In [ ]:
df_result['day_2'] = df_result['day_2'].apply(lambda x: np.nan if x>9 else x)
df_result['day_2'] = df_result['day_2'].astype('Int64')

In [ ]:
sns.histplot(data=df_result, x='day_2')

In [ ]:
sns.histplot(data=df_result, x='day_3')

In [ ]:
df_result['day_3'] = df_result['day_3'].apply(lambda x: np.nan if x>9 else x)
df_result['day_3'] = df_result['day_3'].astype('Int64')

In [ ]:
sns.histplot(data=df_result, x='day_3')

In [ ]:
df_result_daycheck = df_result[~pd.isna(df_result['day_1'])]
df_result_daycheck = df_result_daycheck[~pd.isna(df_result_daycheck['day_2'])]
df_result_daycheck = df_result_daycheck[~pd.isna(df_result_daycheck['day_3'])]
df_result_daycheck['day'] = df_result_daycheck['day_1'].astype(str) + df_result_daycheck['day_2'].astype(str) + df_result_daycheck['day_3'].astype(str)
df_result_daycheck['day'] = df_result_daycheck['day'].astype(int)

In [ ]:
print(len(df_result_daycheck))
df_result_daycheck[['day_1', 'day_2', 'day_3', 'day']].sample(10)

In [ ]:
df_result_nullday = df_result.loc[pd.isna(df_result['day_1']) | pd.isna(df_result['day_2']) | pd.isna(df_result['day_3'])]

In [ ]:
print(len(df_result_nullday))
df_result_nullday[['day_1', 'day_2', 'day_3', 'day']].sample(10)

In [ ]:
df_result_nullday['day'] = np.nan
df_result_nullday[['day_1', 'day_2', 'day_3', 'day']].sample(10)

In [ ]:
df_result2 = pd.concat([df_result_daycheck, df_result_nullday])
print(len(df_result2))
df_result2.head()

#  

#### Check hour digits are < 10, reprocess hour:

In [ ]:
df_result2[['hour_1', 'hour_2', 'hour']].sample(10)

In [ ]:
sns.histplot(data=df_result2, x='hour_1')

In [ ]:
df_result2['hour_1'] = df_result2['hour_1'].apply(lambda x: np.nan if x>9 else x)
df_result2['hour_1'] = df_result2['hour_1'].astype('Int64')

In [ ]:
sns.histplot(data=df_result2, x='hour_1')

In [ ]:
sns.histplot(data=df_result2, x='hour_2')

In [ ]:
df_result2['hour_2'] = df_result2['hour_2'].apply(lambda x: np.nan if x>9 else x)
df_result2['hour_2'] = df_result2['hour_2'].astype('Int64')

In [ ]:
sns.histplot(data=df_result2, x='hour_2')

In [ ]:
df_result_hourcheck = df_result2[~pd.isna(df_result2['hour_1'])]
df_result_hourcheck = df_result_hourcheck[~pd.isna(df_result_hourcheck['hour_2'])]
df_result_hourcheck['hour'] = df_result_hourcheck['hour_1'].astype(str) + df_result_hourcheck['hour_2'].astype(str)
df_result_hourcheck['hour'] = df_result_hourcheck['hour'].astype(int)

In [ ]:
print(len(df_result_hourcheck))
df_result_hourcheck[['hour_1', 'hour_2', 'hour']].sample(10)

In [ ]:
print(len(df_result_hourcheck))
df_result_hourcheck[['hour_1', 'hour_2', 'hour']].sample(10)

In [ ]:
df_result_nullhour = df_result2.loc[pd.isna(df_result2['hour_1']) | pd.isna(df_result2['hour_2'])]

In [ ]:
print(len(df_result_nullhour))
df_result_nullhour[['hour_1', 'hour_2', 'hour']].sample(10)

In [ ]:
df_result3 = pd.concat([df_result_hourcheck, df_result_nullhour])
print(len(df_result3))
df_result3.head()

#  

#### Check minute digits are < 10, reprocess minute:

In [ ]:
df_result3[['minute_1', 'minute_2', 'minute']].sample(10)

In [ ]:
sns.histplot(data=df_result3, x='minute_1')

In [ ]:
df_result3['minute_1'] = df_result3['minute_1'].apply(lambda x: np.nan if x>9 else x)
df_result3['minute_1'] = df_result3['minute_1'].astype('Int64')

In [ ]:
sns.histplot(data=df_result3, x='minute_1')

In [ ]:
sns.histplot(data=df_result3, x='minute_2')

In [ ]:
df_result3['minute_2'] = df_result3['minute_2'].apply(lambda x: np.nan if x>9 else x)
df_result3['minute_2'] = df_result3['minute_2'].astype('Int64')

In [ ]:
sns.histplot(data=df_result3, x='minute_2')

In [ ]:
df_result_minutecheck = df_result3[~pd.isna(df_result3['minute_1'])]
df_result_minutecheck = df_result_minutecheck[~pd.isna(df_result_minutecheck['minute_2'])]
df_result_minutecheck['minute'] = df_result_minutecheck['minute_1'].astype(str) + df_result_minutecheck['minute_2'].astype(str)
df_result_minutecheck['minute'] = df_result_minutecheck['minute'].astype(int)

In [ ]:
print(len(df_result_minutecheck))
df_result_minutecheck[['minute_1', 'minute_2', 'minute']].sample(10)

In [ ]:
df_result_nullminute = df_result3.loc[pd.isna(df_result3['minute_1']) | pd.isna(df_result3['minute_2'])]

In [ ]:
print(len(df_result_nullminute))
df_result_nullminute[['minute_1', 'minute_2', 'minute']].sample(10)

In [ ]:
df_result4 = pd.concat([df_result_minutecheck, df_result_nullminute])
print(len(df_result4))
df_result4.head()

#  

#### Check second digits are < 10, reprocess second:

In [ ]:
df_result4[['second_1', 'second_2', 'second']].sample(10)

In [ ]:
sns.histplot(data=df_result4, x='second_1')

In [ ]:
df_result4['second_1'] = df_result4['second_1'].apply(lambda x: np.nan if x>9 else x)
df_result4['second_1'] = df_result4['second_1'].astype('Int64')

In [ ]:
sns.histplot(data=df_result4, x='second_1')

In [ ]:
sns.histplot(data=df_result4, x='second_2')

In [ ]:
df_result4['second_2'] = df_result4['second_2'].apply(lambda x: np.nan if x>9 else x)
df_result4['second_2'] = df_result4['second_2'].astype('Int64')

In [ ]:
sns.histplot(data=df_result4, x='second_2')

In [ ]:
df_result_secondcheck = df_result4[~pd.isna(df_result4['second_1'])]
df_result_secondcheck = df_result_secondcheck[~pd.isna(df_result_secondcheck['second_2'])]
df_result_secondcheck['second'] = df_result_secondcheck['second_1'].astype(str) + df_result_secondcheck['second_2'].astype(str)
df_result_secondcheck['second'] = df_result_secondcheck['second'].astype(int)

In [ ]:
print(len(df_result_secondcheck))
df_result_secondcheck[['second_1', 'second_2', 'second']].sample(10)

In [ ]:
df_result_nullsecond = df_result4.loc[pd.isna(df_result4['second_1']) | pd.isna(df_result4['second_2'])]

In [ ]:
print(len(df_result_nullsecond))
df_result_nullsecond[['second_1', 'second_2', 'second']].sample(10)

In [ ]:
df_result5 = pd.concat([df_result_secondcheck, df_result_nullsecond])
print(len(df_result5))
df_result5.head()

#  

#### Get year for 'dot' type images:

In [ ]:
df_dot = df_result5.loc[df_result5['processed_image_class'] == 'dot']
print(len(df_dot))
df_dot[['day_1', 'day_2', 'day_3', 'day', 'year']].sample(10)

In [ ]:
sns.histplot(data=df_dot, x='day')

Assuming that 'day' should be between 1 and 365:

In [ ]:
for i in range(0, len(df_dot)):
    day = df_dot['day'].iloc[i]
    if (day < 366) & (day > 0):
        if day > 271:
            df_dot['year'].iloc[i] = 1962
        else:
            df_dot['year'].iloc[i] = 1963

In [ ]:
df_dot[['day_1', 'day_2', 'day_3', 'day', 'year']].sample(10)

In [ ]:
df_not_dot = df_result5.loc[df_result5['processed_image_class'] != 'dot']
print(len(df_not_dot))

In [ ]:
df_result6 = pd.concat([df_dot, df_not_dot])
print(len(df_result6))
df_result6.head()

#  

#### Construct timestamp:¶

In [ ]:
df_result6['day_of_year'] = df_result6['day']
df_result6['day_of_year'] = df_result6['day_of_year'].apply(lambda x: np.nan if x==0 else x)
df_result6['day_of_year'] = df_result6['day_of_year'].apply(lambda x: np.nan if x>365 else x)
df_result6['month_day'] = pd.to_datetime(df_result6['day_of_year'], format='%j').dt.strftime('%m-%d')
df_result6['month'] = df_result6['month_day'].str.split('-').str[0]
df_result6['day'] = df_result6['month_day'].str.split('-').str[1]
df_result6['hour'] = df_result6['hour'].apply(lambda x: np.nan if x>24 else x)
df_result6['minute'] = df_result6['minute'].apply(lambda x: np.nan if x>60 else x)
df_result6['second'] = df_result6['second'].apply(lambda x: np.nan if x>60 else x)
df_result6['Timestamp'] = pd.to_datetime(df_result6[['year', 'month', 'day', 'hour', 'minute', 'second']], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_result6[['year', 'month', 'day', 'hour', 'minute', 'second', 'Timestamp']].sample(10)

#  

#### Check station number digits are < 10, reprocess station number:

In [ ]:
df_result6[['station_number_1', 'station_number_2', 'station_number']].sample(10)

In [ ]:
sns.histplot(data=df_result6, x='station_number_1')

In [ ]:
df_result6['station_number_1'] = df_result6['station_number_1'].apply(lambda x: np.nan if x>9 else x)
df_result6['station_number_1'] = df_result6['station_number_1'].astype('Int64')

In [ ]:
sns.histplot(data=df_result6, x='station_number_1')

In [ ]:
sns.histplot(data=df_result6, x='station_number_2')

In [ ]:
df_result6['station_number_2'] = df_result6['station_number_2'].apply(lambda x: np.nan if x>9 else x)
df_result6['station_number_2'] = df_result6['station_number_2'].astype('Int64')

In [ ]:
sns.histplot(data=df_result6, x='station_number_2')

In [ ]:
df_result_stnnumcheck = df_result6[~pd.isna(df_result6['station_number_1'])]
df_result_stnnumcheck = df_result_stnnumcheck[~pd.isna(df_result_stnnumcheck['station_number_2'])]
df_result_stnnumcheck['station_number'] = df_result_stnnumcheck['station_number_1'].astype(str) + df_result_stnnumcheck['station_number_2'].astype(str)
df_result_stnnumcheck['station_number'] = df_result_stnnumcheck['station_number'].astype(int)

In [ ]:
print(len(df_result_stnnumcheck))
df_result_stnnumcheck[['station_number_1', 'station_number_2', 'station_number']].sample(10)

In [ ]:
df_result_nullstnnum = df_result6.loc[pd.isna(df_result6['station_number_1']) | pd.isna(df_result6['station_number_2'])]

In [ ]:
print(len(df_result_nullstnnum))
df_result_nullstnnum[['station_number_1', 'station_number_2', 'station_number']].sample(10)

In [ ]:
df_result7 = pd.concat([df_result_stnnumcheck, df_result_nullstnnum])
print(len(df_result7))
df_result7.head()

#  

#### Retrieve correct station code:

In [ ]:
df_stn_nums = pd.read_excel(dataDir + 'station_number_changes.xlsx')
df_stn_nums.head()b

Partition results by the change dates, and merge in Station_Code:

In [ ]:
change_dates = df_stn_nums['Start_Date'].drop_duplicates().sort_values().reset_index(drop=True)
change_dates

In [ ]:
stn_nums = df_stn_nums['Station_Number'].unique()
stn_nums

In [ ]:
df_result8 = pd.DataFrame()
for i in range(0, len(change_dates)):
    #Before the first change date - Assume there are no station numbers before launch
    if i == 0:
        print(change_dates[i])
        df_part = df_result7.loc[df_result7['Timestamp'] < change_dates[i]]
        df_result8 = pd.concat([df_result8, df_part])
        print(len(df_result8))
    
    elif i > 0:
        print(change_dates[i])
        df_part_ = df_result7.loc[df_result7['Timestamp'] < change_dates[i]]
        df_part_ = df_part_.loc[df_part_['Timestamp'] >= change_dates[i-1]]
        
        df_sns = pd.DataFrame()
        for sn in stn_nums:
            df_sn = df_stn_nums.loc[df_stn_nums['Station_Number'] == sn]
            df_sn = df_sn.loc[df_sn['Start_Date'] < change_dates[i]]
            if len(df_sn) > 1:
                df_sn = df_sn.sort_values('Start_Date', ascending=False)
                df_sn = df_sn.iloc[:1]
            df_sns = pd.concat([df_sns, df_sn])
        
        df_part = df_part_.merge(df_sns[['Station_Number', 'Station_Code']], how='left', left_on='station_number', right_on='Station_Number')
        df_result8 = pd.concat([df_result8, df_part])
        print(len(df_result8))

#After last change date
last_change_date = change_dates[len(change_dates)-1]
print('After ' + str(last_change_date))
df_part_ = df_result7.loc[df_result7['Timestamp'] > last_change_date]

df_sns = pd.DataFrame()
for sn in stn_nums:
    df_sn = df_stn_nums.loc[df_stn_nums['Station_Number'] == sn]
    if len(df_sn) > 1:
        df_sn = df_sn.sort_values('Start_Date', ascending=False)
        df_sn = df_sn.iloc[:1]
    df_sns = pd.concat([df_sns, df_sn])

df_part = df_part_.merge(df_sns[['Station_Number', 'Station_Code']], how='left', left_on='station_number', right_on='Station_Number')
df_result8 = pd.concat([df_result8, df_part])
print(len(df_result8))

In [ ]:
print(len(df_result8))
df_result8[['station_number', 'Station_Number', 'Timestamp', 'Station_Code']].sample(10)

Add part with null Timestamp:

In [ ]:
df_part_null = df_result7.loc[pd.isna(df_result7['Timestamp'])]
print(len(df_part_null))
df_part_null.head()

In [ ]:
df_result8 = pd.concat([df_result8, df_part_null])

In [ ]:
print(len(df_result8))
df_result8[['station_number', 'Station_Number', 'Timestamp', 'Station_Code']].sample(10)

#  

#### Get station details:¶

In [1]:
df_stns = pd.read_csv(dataDir + 'ground_stations.csv')
df_stns.head()

NameError: name 'pd' is not defined

In [ ]:
df_result9 = df_result8.merge(df_stns[['Station_Code', 'Station_Name', 'Lat', 'Lng']], how='left', on='Station_Code')

In [ ]:
print(len(df_result9))
df_result9[['Station_Number', 'Timestamp', 'Station_Code', 'Station_Name', 'Lat', 'Lng']].sample(10)

#  

#### Save:

In [ ]:
df_result9 = df_result9.sort_values('Timestamp').reset_index(drop=True)
df_result9.to_csv(resultDir + 'result_total2.csv', index=False)